In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import json 
import argparse 
import sys
from random import  uniform
from itertools import combinations
import rmab.secret
import rmab.fr_dynamics 
import rmab.database

In [3]:
num_trials = 100
num_points = 1000
K = 4


## Looking at Match Probability

## Trying out Tries

In [4]:
from collections import defaultdict

class TrieNode:
    def __init__(self):
        self.children = defaultdict(TrieNode)
        self.max_value = float('-inf')

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, nums, value):
        node = self.root
        for num in nums:
            node = node.children[num]
            node.max_value = max(node.max_value, value)

class SegmentTree:
    def __init__(self, n):
        self.tree = [float('-inf')] * (4 * n)

    def update(self, idx, start, end, pos, val):
        if start == end:
            self.tree[idx] = val
        else:
            mid = (start + end) // 2
            if pos <= mid:
                self.update(2 * idx + 1, start, mid, pos, val)
            else:
                self.update(2 * idx + 2, mid + 1, end, pos, val)
            self.tree[idx] = max(self.tree[2 * idx + 1], self.tree[2 * idx + 2])

    def query(self, idx, start, end, left, right):
        if start > right or end < left:
            return float('-inf')
        if left <= start and end <= right:
            return self.tree[idx]
        mid = (start + end) // 2
        return max(self.query(2 * idx + 1, start, mid, left, right),
                   self.query(2 * idx + 2, mid + 1, end, left, right))

def build_data_structure(lists):
    trie = Trie()
    for lst, value in lists:
        trie.insert(lst, value)
    return trie

def query_max(trie, segment_trees, query_list):
    node = trie.root
    for num in query_list:
        if num not in node.children:
            return float('-inf')
        node = node.children[num]
    return segment_trees[len(query_list)].query(0, 0, len(query_list) - 1, 0, len(query_list) - 1)

# Example usage
lists = [([1, 2, 3], 10), ([1, 3, 5], 20), ([2, 4, 6], 30)]
trie = build_data_structure(lists)
segment_trees = {}
for i in range(1, max(len(lst) for lst, _ in lists) + 1):
    segment_trees[i] = SegmentTree(i)
    for lst, value in lists:
        if len(lst) >= i:
            segment_trees[i].update(0, 0, i - 1, i - 1, value)

query_list = [1, 3]
print(query_max(trie, segment_trees, query_list))  # Output: 20


30


## Mathematica Generation

In [4]:
def all_subsets(n):
    elements = set(range(1, n + 1))
    all_subsets_list = []

    for subset_size in range(len(elements) + 1):
        all_subsets_list.extend(set(combinations(elements, subset_size)))

    return all_subsets_list


In [5]:
def get_all_subsets_given_subset(input_subset):
    if len(input_subset) == 0:
        return [()]

    n = max(input_subset)
    elements = set(range(1, n + 1))
    all_subsets_list = []

    for subset_size in range(len(elements) + 1):
        for subset in combinations(elements, subset_size):
            if set(subset).issubset(input_subset):
                all_subsets_list.append(subset)

    return all_subsets_list


In [6]:
def subset_to_var(subset,letter):
    if len(subset) == 0:
        return "0"
    variables = sorted(list(subset))
    variables = "".join([str(i) for i in variables])
    return "{}{}".format(letter,variables)

In [13]:
num_points = 7
all_variables = set()
constraint_list = []
for letter in ["U","V"]:
    for i in range(1,num_points+1):
        constraint_list.append("0 <= {}{} <= 1".format(letter,i))

    for B in all_subsets(num_points):
        if len(B) > 0:
            all_variables.add(subset_to_var(B,letter))
        for A in get_all_subsets_given_subset(B):
            if A == B:
                continue 
        
            # Monotonicity 
            if len(A) > 0:
                constraint_list.append("{} >= {}".format(subset_to_var(B,letter),subset_to_var(A,letter)))
            
            for j in range(1,num_points+1):
                A_with = list(A)
                A_with.append(j)
                A_with = sorted(list(set(A_with)))

                B_with = list(B)
                B_with.append(j)
                B_with = sorted(list(set(B_with)))

                if list(B_with) == list(B):
                    continue 
                if list(A_with) == list(B_with):
                    constraint_list.append("{} >= {}".format(subset_to_var(B,letter),subset_to_var(A,letter)))
                elif subset_to_var(A,letter) == "0":
                    constraint_list.append("{} >= {} - {}".format(subset_to_var(A_with,letter),subset_to_var(B_with,letter),subset_to_var(B,letter)))
                else:
                    constraint_list.append("{} - {} >= {} - {}".format(subset_to_var(A_with,letter),subset_to_var(A,letter),subset_to_var(B_with,letter),subset_to_var(B,letter)))

# flipped_points = [1,3]

# for i in range(1,num_points+1):
#     if i!=3:
#         other_points = [j for j in flipped_points if j!=3]
#         if 3 not in flipped_points:
#             other_points += [j]
        
#         other_side = [j for j in flipped_points if j!=i]
#         if i not in flipped_points:
#             other_side += [i]

#         constraint_list.append("U3 + {} >= U{} + {}".format(subset_to_var(other_points,"V"),i,subset_to_var(other_side,"V")))

#         if i!=4:
#             other_points = [j for j in flipped_points if j!=3 and j!=4]
#             if 3 not in flipped_points:
#                 other_points += [3]
#             if 4 not in flipped_points:
#                 other_points += [4]
            
#             other_side_point = [3,i]
#             other_side = [j for j in flipped_points if j!=i and j!=3]
#             if 3 not in flipped_points:
#                 other_side += [3]
#             if i not in flipped_points:
#                 other_side += [i]

#             constraint_list.append("U34 + {} >= {} + {}".format(subset_to_var(other_points,"V"),subset_to_var(other_side_point,"U"),subset_to_var(other_side,"V")))

#             if i!=5:
#                 other_points = [j for j in flipped_points if j!=3 and j!=4 and j!=5]
#                 if 3 not in flipped_points:
#                     other_points += [3]
#                 if 4 not in flipped_points:
#                     other_points += [4]
#                 if 5 not in flipped_points:
#                     other_points += [5]
                
#                 other_side_point = [3,4,i]
#                 other_side = [j for j in flipped_points if j!=i and j!=3 and j!=4]
#                 if 3 not in flipped_points:
#                     other_side += [3]
#                 if 4 not in flipped_points:
#                     other_side += [4]
#                 if i not in flipped_points:
#                     other_side += [i]

#                 constraint_list.append("U345 + {} >= {} + {}".format(subset_to_var(other_points,"V"),subset_to_var(other_side_point,"U"),subset_to_var(other_side,"V")))
print(str(list(all_variables)).replace("'",'').replace("[","{").replace("]","}"))
print(str(constraint_list).replace("'",'').replace("[","{").replace("]","}"))

{U123456, U23456, V124, U13567, U37, U237, U13457, V236, V1245, V14567, U36, V234567, V57, V256, V167, V267, V1234567, U1236, U1235, U137, U3456, V245, V367, V1367, U1234, V56, V12456, V123457, V14, V123467, V1246, V1236, U15, U23567, U14567, V2, V2457, V1256, U12467, V45, V2347, V23457, V3, V13457, V157, U3467, V3467, U67, V2456, V2356, U3567, V467, U12367, U123567, V23, V12347, U12357, U24567, V13, U457, U2357, V12467, V134567, U123, U26, U357, V17, V4, V12457, U2567, V2467, V12345, U367, U123457, U256, V2345, V1345, U124, U1457, V1567, V137, U2, V13567, U1347, V35, V156, U1357, U1345, U12457, V5, U2345, V1247, U46, U2346, V23467, V3567, V47, V234, V1237, U12356, U2347, U13467, U134567, U2367, V2567, U23467, V1467, U127, U136, V36, V4567, U2457, U12456, V456, V1257, V1457, V6, V3456, U1356, V12356, U56, U3, V25, V34, U126, V345, U12, U1367, U1456, U45, V257, U4, V346, U1247, V1357, U147, V27, V246, U2467, V567, V124567, U1245, U134, U124567, V457, V134, V12367, U3457, U1234567, V1235

{V1345 -> 31/28, U135 -> 3/28, U123 -> 1/7, V15 -> 15/14, 
 V13 -> 29/28, U345 -> 1/14, U12 -> 1/7, U145 -> 3/28, U1245 -> 1/7, 
 V23 -> 29/28, V234 -> 31/28, U1235 -> 1/7, U25 -> 3/28, V14 -> 29/28,
  U12345 -> 1/7, U14 -> 3/28, V3 -> 1, V34 -> 31/28, U234 -> 3/28, 
 V35 -> 15/14, V245 -> 15/14, V145 -> 15/14, U24 -> 3/28, 
 V12 -> 11/42, V1235 -> 31/28, V24 -> 29/28, U5 -> 1/14, U2 -> 1/14, 
 V123 -> 15/14, V1245 -> 31/28, V2 -> 19/84, U23 -> 3/28, 
 V135 -> 31/28, V45 -> 29/28, U1345 -> 3/28, U245 -> 3/28, 
 V345 -> 31/28, U1234 -> 1/7, V1 -> 19/84, U13 -> 3/28, U45 -> 1/14, 
 U2345 -> 3/28, V124 -> 15/14, U35 -> 1/14, U134 -> 3/28, U125 -> 1/7,
  U4 -> 1/14, V134 -> 31/28, U3 -> 1/14, V235 -> 31/28, U235 -> 3/28, 
 U15 -> 3/28, V4 -> 1, V12345 -> 31/28, U1 -> 1/14, U124 -> 1/7, 
 V125 -> 31/28, V1234 -> 31/28, V5 -> 1, V2345 -> 31/28, V25 -> 15/14,
  U34 -> 1/14}

In [14]:
len(constraint_list)

13188

## Testing out Optimism

In [24]:
num_points = 2
K = 2
p_values = [random.random() for i in range(num_points)]
p_values = sorted(p_values,reverse=True)
p_values = [1,1]
l = [random.randint(0,1) for i in range(num_points)]
l = [1,1]
states = []
for i in range(num_points):
    if l[i] == 1:
        states.append(1)
    else:
        states.append(random.randint(0,1))
v_values = []
discount = 0.9
for i in range(num_points):
    if l[i] == 0 and states[i] == 1:
        v_values.append(p_values[i])
    elif l[i] == 1:
        v_values.append(p_values[i]/(1-discount))
    else:
        v_values.append(0)
s = set()
for i in range(num_points):
    if len(s) == K:
        break 
    elif l[i] == 1:
        s.add(i)
    elif l[i] == 0 and states[i] == 1:
        v = np.prod(1-np.array([p_values[j] for j in s]))
        v*=(p_values[i])
        vf = np.prod(1-np.array([p_values[j] for j in s if l[j] == 1]))
        vf *= (p_values[i])
        if v > vf * discount:
            s.add(i)
best_val = -1
best_combo = ()

for c in combinations(range(num_points),K):
    immediate_value = 1-np.prod(1-np.array([p_values[j]*states[j] for j in c]))
    future_value = 0
    for j in range(num_points):
        if l[j] == 1:
            future_value += v_values[j] 
        elif l[j] == 0 and states[j] == 1 and j not in c:
            future_value += v_values[j] 
    total_value = immediate_value + discount*future_value 
    if total_value > best_val:
        best_val = total_value 
        best_combo = c  

In [25]:
s, 1-np.prod(1-np.array([p_values[j] for j in s])), states, l 

({0, 1}, 1, [1, 1], [1, 1])

In [21]:
best_combo, 1-np.prod(1-np.array([p_values[j] for j in best_combo]))

((0, 1, 2), 0.7752768911725616)

In [9]:
states  

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [5]:

if len(points)<10:
    combinations_list = list(combinations(points, K))

    scores = []
    for combo in combinations_list:
        combo = np.array(combo)
        matching_score = 1-np.prod(combo[:,0])
        engagement_score =  np.sum(combo[:,1])
        scores.append(matching_score + engagement_score)
    np.max(scores)

In [6]:
epsilon = .01 
dp = np.zeros((K+1,num_points+1,round(1/epsilon)+1))

In [7]:
for k in range(dp.shape[0]):
    for n in range(dp.shape[1]):
        for e in range(dp.shape[2]):
            if k == 0 or n == 0:
                dp[k][n][e] = 1-e*epsilon
            else: 
                baseline = dp[k][n-1][e]
                other_option = dp[k-1][n-1][round(e*points[n-1][0])] + points[n-1][1]
                dp[k][n][e] = max(baseline,other_option)

In [8]:
dp[-1][-1][-1]

4.995494652146921

In [14]:
def to_prob(x):
    x = np.maximum(x,0)
    return (100-x)/100

num_times = 1000

mu_values = np.array([0.0 for i in range(num_points)])
max_vals = np.array([0.0 for i in range(num_points)])
num_pulls = np.array([0.0 for i in range(num_points)])
observed_vals = np.array([0.0 for i in range(num_points)])

upper_bound_match = np.prod(sorted(points[:,0])[-K-1:])
upper_bound_engagement = np.sum(sorted(points[:,1])[-K-1:])

for i in range(len(mu_values)):
    max_val = 1-points[i][0]*upper_bound_match+points[i][1]+upper_bound_engagement
    max_vals[i] = max_val 
    mu_values[i] = max_val 

for i in range(num_times):
    current_cohort = np.argsort(mu_values)[-k:][::-1]
    real_val = 1-np.prod(points[current_cohort,0]) + np.sum(points[current_cohort,1])
    observed_vals[current_cohort] = np.maximum(observed_vals[current_cohort],real_val)
    num_pulls[current_cohort] += 1

    probs = to_prob(num_pulls[current_cohort])
    mu_values[current_cohort] = probs * max_vals[current_cohort] + (1-probs)*observed_vals[current_cohort]
cohort_vals = []
for i in range(num_times):
    rand_points = points[np.random.choice(points.shape[0], k, replace=False)]
    real_val = 1-np.prod(rand_points[:,0]) + np.sum(rand_points[:,1])
    cohort_vals.append(real_val)
np.max(observed_vals), np.max(cohort_vals)

(4.958504375312464, 4.462580151919658)

In [21]:
greedy_vals = [(1-i[0]+i[1]) for i in points]
max_greedy = np.argsort(greedy_vals)[-k:][::-1]
true_points = points[max_greedy]
score = 1-np.prod(true_points[:,0])+np.sum(true_points[:,1])
score

4.8985039670757216